In [39]:
import sqlite3

conn = sqlite3.connect('dogs.db', timeout=10)

c = conn.cursor()

c.execute('DROP TABLE IF EXISTS dogimages')


In [8]:
import os
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [40]:
c.execute('''
          CREATE TABLE dogimages (
            id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            breed TEXT,
            image BLOB
          )''')

conn.commit()


In [30]:
current_directory = "/content/gdrive/MyDrive/Project_4/stanford_dogs/cropped/test/"

contents = os.listdir(current_directory)

for item in contents:

    directory = f"/content/gdrive/MyDrive/Project_4/stanford_dogs/cropped/test/{item}"
    contents_sub_dir = os.listdir(directory)
    name = item.split("-",1)[1]
    for items in contents_sub_dir:
     # print("/content/gdrive/MyDrive/Project_4/stanford_dogs/cropped/test/" + item + items)
      #print(name)
      full_path = "/content/gdrive/MyDrive/Project_4/stanford_dogs/cropped/test/" + item +"/"+ items
      #query = "INSERT INTO dogimages(breed,image) VALUES(%s, LOAD_FILE(%s))"
      query = "INSERT INTO dogimages VALUES(NULL,?,?)"
      values = (name, full_path)
      #print(query)
      c.execute(query, values)

conn.commit()

In [ ]:
c.execute("SELECT * FROM dogimages LIMIT 100")

my_table = c.fetchall()

for i in my_table:
    print(i)

In [ ]:
def convertToBinaryData(filename):
    # Convert digital data to binary format
    with open(filename, 'rb') as file:
        blobData = file.read()
    return blobData

def insertBLOB(breed, image):
    try:
        sqliteConnection = sqlite3.connect('dogs.db', timeout=10)
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")
        sqlite_insert_blob_query = """ INSERT INTO dogimages
                                  (breed, image) VALUES (?, ?)"""

        empPhoto = convertToBinaryData(image)

        # Convert data into tuple format
        data_tuple = (breed, image)
        cursor.execute(sqlite_insert_blob_query, data_tuple)
        sqliteConnection.commit()
        print("Image and file inserted successfully as a BLOB into a table")
        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert blob data into sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("the sqlite connection is closed")

current_directory = "/content/gdrive/MyDrive/Project_4/stanford_dogs/cropped/test/"

contents = os.listdir(current_directory)

for item in contents:

    directory = f"/content/gdrive/MyDrive/Project_4/stanford_dogs/cropped/test/{item}"
    contents_sub_dir = os.listdir(directory)
    name = item.split("-",1)[1]
    for items in contents_sub_dir:
      full_path = "/content/gdrive/MyDrive/Project_4/stanford_dogs/cropped/test/" + item +"/"+ items
      insertBLOB(name, full_path)


conn.commit()

In [ ]:
def writeTofile(data, filename):
    # Convert binary data to proper format and write it on Hard Disk
    with open(filename, 'wb') as file:
        file.write(data)
    print("Stored blob data into: ", filename, "\n")

def readBlobData(breed):
    try:
        sqliteConnection = sqlite3.connect('dogs.db')
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        sql_fetch_blob_query = """SELECT * dogimages where breed = ?"""
        cursor.execute(sql_fetch_blob_query, (breed,))
        record = cursor.fetchall()
        for row in record:
            print("breed = ", row[0], "image = ", row[1])
            name = row[1]
            photo = row[2]


            print("Storing employee image and resume on disk \n")
            photoPath = "E:\pynative\Python\photos\db_data\\" + name + ".jpg"

            writeTofile(photo, photoPath)


        cursor.close()

    except sqlite3.Error as error:
        print("Failed to read blob data from sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("sqlite connection is closed")


In [ ]:
conn.close()